# 1. Setup     

In [1]:
import os, sys
import re
import datetime
import pandas as pd

import geopandas as gpd

from osgeo import gdal
from osgeo import ogr
from osgeo import osr

import zipfile

Set environment variables and globals to help parse directory to find input data

In [2]:
# For Windows OS
os.environ["HOME"] = os.path.expanduser("~")
CURRENT_DIR = os.getcwd()

# add Environment Variable PATH to bin executables
bin_path = os.environ["HOME"] + '\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin'
os.environ["PATH"] = r"{};{}".format(os.environ["PATH"], bin_path)

# add path to arcpy lib from ArcGIS Pro
arcpy_path = os.environ["HOME"] + '\\AppData\\Local\\Programs\\ArcGIS\\Pro\\Resources\\ArcPy'
sys.path.append(arcpy_path)


Instantiate ArcGIS PRO

In [3]:
import arcgis
from arcgis.gis import GIS
gis = GIS('pro')

<b> Input: </b>
<li>1a. list of employee names to include in the output</li>
<li>1b. start date and end dates as datetime.datetime objects</li>
<li>1c. the folder that contains the input files</li>
<li>1d. the name of the output shapefile (include your initials or ID in the name to make the name unique for uploading it to AGOL)</li>
<li>1e. the name of the output csv file </li>

<b>1a. List of Employee Names</b>

In [4]:
employeeList = ['Jones, Frank','Brown, Justine', 'Samulson, Roger']

<b>1b. Start and End dates as datetime obj</b>

In [5]:
startDate = datetime.datetime(2016, 6, 26)
endDate = datetime.datetime(2017, 5, 11)

<b>1c. Specify Input Filepath: zip file with travel + employee Data</b>

In [6]:
IN_filepath = os.path.join(CURRENT_DIR, "input_data")

In [7]:
OUT_filepath = os.path.join(CURRENT_DIR, "results")

Create Generic Output filename to append .zip, .shp, and .csv

In [8]:
output_fn = os.path.join(OUT_filepath, ("employee_trips_" + gis.users.me.username.split('_')[0]))
output_zip = output_fn + '.zip'

<b>1d. Name of Output Shape file</b>

In [9]:
output_shape_fn = output_fn + '.shp'

<b>1e. Name of Output CSV file</b>

In [10]:
output_csv = output_fn + '.csv'

View Inputs:

In [11]:
print("List of Employees:\t",employeeList)
print("Start Date:\t\t", startDate)
print("End Date:\t\t", endDate, "\n")
print("Input Directory:\n", IN_filepath,"\n")
print("Output Shapefile:\t", os.path.basename(output_shape_fn))
print("Output CSV:\t\t", os.path.basename(output_csv), "\n")

List of Employees:	 ['Jones, Frank', 'Brown, Justine', 'Samulson, Roger']
Start Date:		 2016-06-26 00:00:00
End Date:		 2017-05-11 00:00:00 

Input Directory:
 C:\Users\blcrosbie\dev\GIS\PSU_GEOG_489\L3\input_data 

Output Shapefile:	 employee_trips_blc72.shp
Output CSV:		 employee_trips_blc72.csv 



<b>Extra: specify duration max/min</b>

In [37]:
# Extra component, input range of duration
max_duration = 10
min_duration = 3

# 2. Load Employee Data

In [12]:
emp_fn = os.path.join(IN_filepath, "employees.csv" )
employees = pd.read_csv(emp_fn, header=None)

Check that the employee data has been loaded into the dataframe
<li>(edit: header=None, since there is no column names set)
<li> then set the column names 

In [13]:
employees.columns = ['Name', 'ID']

In [14]:
employees = employees.astype({'Name': 'str', 'ID': 'int32'})
employees.dtypes

Name    object
ID       int32
dtype: object

In [15]:
employees.head()

,Name,ID
0,"Smith, Richard",1234421
1,"Moore, Lisa",1231233
2,"Jones, Frank",2132222
3,"Brown, Justine",2132225
4,"Samulson, Roger",3981232


# 3. Aggregate Travel Data

In [16]:
travel_pattern = "travel_\d\d\d\d.csv$"
travel_file_list = []

for root, dirs, files in os.walk(IN_filepath):
    travel_file_list = [fn for fn in files if re.match(travel_pattern, fn)]

Check first 3 and last 3 and length

In [17]:
print(travel_file_list[:3])
print(travel_file_list[-3:])
print(len(travel_file_list))

['travel_1001.csv', 'travel_1002.csv', 'travel_1003.csv']
['travel_1073.csv', 'travel_1074.csv', 'travel_1075.csv']
75


Parse List and concatenate into single DataFrame

In [18]:
travel = pd.DataFrame(columns=['ID','Start', 'End', 'Route'], dtype=object)

In [19]:
for fn in travel_file_list:
    trip_file = os.path.join(IN_filepath, fn)
    trip = pd.read_csv(trip_file, header=None, dtype=object)
    trip.columns = ['ID', 'Start', 'End', 'Route']
    travel = pd.concat([travel, trip], axis=0)


Check to make sure Travel dataframe is valid

In [20]:
travel = travel.astype({'ID': 'int32'})
travel.head()

,ID,Start,End,Route
0,2132222,2016-01-07 16:00:00,2016-01-26 12:00:00,Cleveland;Bangor;Erie;Philadelphia;New York;Al...
0,1234421,2016-01-15 13:00:00,2016-01-31 17:00:00,Philadelphia;Portland;Harrisburg;Burlington;Er...
0,2132225,2016-01-29 12:00:00,2016-02-03 15:00:00,Bangor;Cleveland;Augusta
0,2132225,2016-02-10 07:00:00,2016-02-22 22:00:00,Altoona;Augusta;Altoona;Bangor;Augusta;Columbus
0,2132222,2016-02-19 14:00:00,2016-02-25 13:00:00,Boston;Philadelphia


# 4. Merge Employee and Travel Data

In [22]:
employee_travel = employees.merge(travel, on='ID')

In [23]:
employee_travel.head()

,Name,ID,Start,End,Route
0,"Smith, Richard",1234421,2016-01-15 13:00:00,2016-01-31 17:00:00,Philadelphia;Portland;Harrisburg;Burlington;Er...
1,"Smith, Richard",1234421,2016-04-09 06:00:00,2016-04-16 22:00:00,Augusta;New York;Columbus;Syracuse;Albany
2,"Smith, Richard",1234421,2016-04-22 15:00:00,2016-05-02 20:00:00,Columbus;Burlington;Augusta;Syracuse;Augusta;P...
3,"Smith, Richard",1234421,2016-05-09 06:00:00,2016-05-18 22:00:00,Boston;Albany;Harrisburg;Erie
4,"Smith, Richard",1234421,2016-05-19 08:00:00,2016-05-25 16:00:00,Portland;Washington


<b>Check Column Datatypes, make sure datetime formatted correctly </b>

In [24]:
employee_travel.dtypes

Name     object
ID        int32
Start    object
End      object
Route    object
dtype: object

In [25]:
employee_travel = employee_travel.astype({'Name': str, 'Route': str})

In [26]:
employee_travel['Start'] = pd.to_datetime(employee_travel['Start'], format="%Y-%m-%d %H:%M:%S")
employee_travel['End'] = pd.to_datetime(employee_travel['End'], format="%Y-%m-%d %H:%M:%S")

In [27]:
employee_travel.dtypes

Name             object
ID                int32
Start    datetime64[ns]
End      datetime64[ns]
Route            object
dtype: object

<b>Last pandas.dtypes Validation should show:</b>
<li> Name: string or object </li>
<li> ID: int32 </li>
<li> Start: datetime64[ns] </li>
<li> End: datetime64[ns] </li>
<li> Route: string or object </li>

<b> 4a. Filter using input Employee List, and start-end dates </b>

In [28]:
# copy original dataframe 
my_trips_df = employee_travel.copy()

# filter Name(s) from list
my_trips_df = my_trips_df[my_trips_df['Name'].isin(employeeList)]

# filter by date
my_trips_df = my_trips_df[my_trips_df['Start'] >= startDate]# & 
my_trips_df = my_trips_df[my_trips_df['End'] <= endDate]

<b> 4b. Calculate Trip Duration </b>

In [29]:
# Calculate duration
my_trips_df['Duration'] = my_trips_df[['Start', 'End']].apply(lambda x: (x[1]-x[0]).days, axis=1)

<b> 4c. Reorder Columns and Reset Index</b>
<li>Columns [Name, ID, Duration, Start, End, Route]</li>
<li>Reset the index (row labels being integer 0, 1, 2...</li>

In [30]:
# column reorder
reorder_cols = ['Name', 'ID', 'Duration', 'Start', 'End', 'Route']
my_trips_df = my_trips_df[reorder_cols]

<b>4d. Sort DataFrame</b>
<li>Name (alphabetical)</li>
<li>Duration from shortest to longest (ascending=True)</li>
<li>Start from earliest to most recent (ascending=True)</li>
<li>End from earliest to most recent (ascending=True)</li>

In [31]:
my_trips_df = my_trips_df.sort_values(by=['Name', 'Duration', 'Start', 'End'], ascending=True)

Reset Index and View Results

In [33]:
# Reset the index like example (start from 0)
my_trips_df = my_trips_df.reset_index(drop=True)
my_trips_df.head()

,Name,ID,Duration,Start,End,Route
0,"Brown, Justine",2132225,5,2016-11-30 16:00:00,2016-12-06 14:00:00,Pittsburgh;Syracuse
1,"Brown, Justine",2132225,9,2016-12-12 12:00:00,2016-12-21 19:00:00,Columbus;Boston;Pittsburgh
2,"Brown, Justine",2132225,12,2016-08-02 13:00:00,2016-08-14 16:00:00,Albany;Scranton;Philadelphia;Scranton;Augusta
3,"Brown, Justine",2132225,12,2017-01-07 11:00:00,2017-01-19 17:00:00,Harrisburg;Portland;Boston;Syracuse;Albany
4,"Brown, Justine",2132225,17,2017-03-27 06:00:00,2017-04-13 14:00:00,Portland;Boston;Bangor;Washington;Harrisburg;P...


# 5. Save to CSV:
to file Output_CSV

In [34]:
my_trips_df.to_csv(output_csv)

<b> Extra: filter on duration and save separate file </b>

In [39]:
duration_filtered_df = my_trips_df.copy()
duration_filtered_df = duration_filtered_df[\
                            (duration_filtered_df['Duration'] >= min_duration)\
                                            &\
                            (duration_filtered_df['Duration'] <= max_duration)]

C:\Users\blcrosbie\dev\GIS\PSU_GEOG_489\L3\results\employee_trips_blc72.csv


In [43]:
def get_extra_filename(filename):
    return filename.split('.')[0] + '_extra' + '.' + filename.split('.')[1]

In [45]:
csv_extra_fn = get_extra_filename(output_csv)
duration_filtered_df.to_csv(csv_extra_fn)

# 6. Create WGS84 coordinates with GeoPandas

Import the neCities Shapefile into a GeoPandas DataFrame

In [46]:
my_neCities_file = "ne_cities_" + gis.users.me.username.split('_')[0]
my_neCities_shp = os.path.join(IN_filepath, "ne_cities.shp")

In [47]:
ne_Cities_df = gpd.read_file(my_neCities_shp)

In [48]:
ne_Cities_df.head()

,OBJECTID,UIDENT,POPCLASS,NAME,CAPITAL,STATEABB,COUNTRY,geometry
0,15,62507.0,2.0,Bangor,-1.0,US-ME,USA,POINT (-68.77619 44.81189)
1,20,64707.0,2.0,Waterville,-1.0,US-ME,USA,POINT (-69.62073 44.53724)
2,23,65607.0,2.0,Augusta,1.0,US-ME,USA,POINT (-69.78141 44.32166)
3,26,66707.0,2.0,Berlin,-1.0,US-NH,USA,POINT (-71.19324 44.46470)
4,29,67407.0,2.0,Lewiston,-1.0,US-ME,USA,POINT (-70.19406 44.10972)


In [49]:
# translate city names to Coordinates
def get_coordinates(route, shape_df):
    route = route.split(';')
    points = [','.join([ '{0} {1}'.format(\
                shape_df[shape_df.NAME == city].geometry.x.iloc[0],\
                shape_df[shape_df.NAME == city].geometry.y.iloc[0])]) for city in route]
    
    wkt = 'LineString (' + ','.join(points) + ')'

    return wkt

In [50]:
my_trips_df['wkt'] = my_trips_df['Route'].apply(lambda x: get_coordinates(x, ne_Cities_df))
my_trips_df.head()

,Name,ID,Duration,Start,End,Route,wkt
0,"Brown, Justine",2132225,5,2016-11-30 16:00:00,2016-12-06 14:00:00,Pittsburgh;Syracuse,LineString (-79.91439598338775 40.447399277754...
1,"Brown, Justine",2132225,9,2016-12-12 12:00:00,2016-12-21 19:00:00,Columbus;Boston;Pittsburgh,LineString (-82.98182532058195 39.972384950705...
2,"Brown, Justine",2132225,12,2016-08-02 13:00:00,2016-08-14 16:00:00,Albany;Scranton;Philadelphia;Scranton;Augusta,LineString (-73.78191599152962 42.674015577157...
3,"Brown, Justine",2132225,12,2017-01-07 11:00:00,2017-01-19 17:00:00,Harrisburg;Portland;Boston;Syracuse;Albany,LineString (-76.82629734234831 40.266819768117...
4,"Brown, Justine",2132225,17,2017-03-27 06:00:00,2017-04-13 14:00:00,Portland;Boston;Bangor;Washington;Harrisburg;P...,LineString (-70.27758484016806 43.698554552950...


# 7. Use GDAL/OGR to create Shapefile
Create Geometry from LineStrings Using ogr</b>
<li> first method with wkt.loads works if trying to use GeoPandas to save shapefile</li>
<li> use shapely wkt.loads even though output is seemingly identical</li>

In [51]:
# my_trips_df['geometry'] = my_trips_df['wkt'].apply(wkt.loads)
my_trips_df['geometry'] = my_trips_df['wkt'].apply(lambda x: ogr.CreateGeometryFromWkt(x))

<b> Create Projection, Spatial Ref, and Build layer Fields </b>

In [52]:
# Create the Projection, get ESRI driver
drv = ogr.GetDriverByName('ESRI Shapefile')

# Make Spatial Ref Container
sr = osr.SpatialReference() # create spatial reference
sr.ImportFromEPSG(4326) # set it to EPSG:4326

# Now initialize the Shapefile and Build the layer class
outfile = drv.CreateDataSource(output_shape_fn)
outlayer = outfile.CreateLayer('Employee_Trips', geom_type=ogr.wkbLineString, srs=sr)

# now create fields in output Layer
employeeField = ogr.FieldDefn('Name', ogr.OFTString)
outlayer.CreateField(employeeField)
employeeIDField = ogr.FieldDefn('ID', ogr.OFTInteger)
outlayer.CreateField(employeeIDField)
tripLen = ogr.FieldDefn('Duration', ogr.OFTInteger)
outlayer.CreateField(tripLen)
tripStart = ogr.FieldDefn('Start', ogr.OFTString)
outlayer.CreateField(tripStart)
tripEnd = ogr.FieldDefn('End', ogr.OFTString)
outlayer.CreateField(tripEnd)
tripRoute = ogr.FieldDefn('Route', ogr.OFTString)
outlayer.CreateField(tripRoute)

featureDefn = outlayer.GetLayerDefn()

Validate the feature Definition in output layer

In [53]:
for i in range(featureDefn.GetFieldCount()):
    print(featureDefn.GetFieldDefn(i).GetName())

Name
ID
Duration
Start
End
Route


<b> Populate Output Layer from the Employee Travel DataFrame </b>

In [54]:
for row in my_trips_df.index:
    geometry = my_trips_df.loc[row, 'geometry']
    name = my_trips_df.loc[row, 'Name']
    this_ID = int(my_trips_df.loc[row, 'ID'])
    duration = int(my_trips_df.loc[row, 'Duration'])
    start = str(my_trips_df.loc[row, 'Start'])
    end = str(my_trips_df.loc[row, 'End'])
    route = my_trips_df.loc[row, 'Route']
    
    outFeature = ogr.Feature(featureDefn)
    outFeature.SetGeometry(geometry)
    outFeature.SetField('Name', name)
    outFeature.SetField('ID', this_ID)
    outFeature.SetField('Duration', duration)
    outFeature.SetField('Start', start)
    outFeature.SetField('End', end)
    outFeature.SetField('Route', route)
    outlayer.CreateFeature(outFeature)
    outFeature = None
    

Validate Ouput layer is being populated by checking the Routes

In [55]:
for feature in outlayer:
    print(feature.GetField('Route'))
outlayer.ResetReading()   

Pittsburgh;Syracuse
Columbus;Boston;Pittsburgh
Albany;Scranton;Philadelphia;Scranton;Augusta
Harrisburg;Portland;Boston;Syracuse;Albany
Portland;Boston;Bangor;Washington;Harrisburg;Pittsburgh
New York;Portland;Boston;Portland;Washington;Pittsburgh;Syracuse;Scranton;Boston
Harrisburg;Augusta
Philadelphia;Boston;Altoona;Harrisburg;Scranton
Augusta;New York;Scranton;Columbus;Washington;Augusta;Bangor
Boston;Syracuse;Portland;Altoona
Portland;Augusta;Burlington;Philadelphia
Erie;Syracuse;Philadelphia;Bangor;Scranton
New York;Augusta;Pittsburgh;Cleveland;Altoona
Boston;Cleveland;Pittsburgh;Philadelphia;Harrisburg;Albany;Portland;Harrisburg


In [56]:
# close file 
outfile = None

# 8. Zip files together to publish shapefile

In [57]:
my_zipped_shape = zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED)
RESULTS_DIR = os.path.join(CURRENT_DIR , "results\\")
just_fn = os.path.basename(output_zip).split('.')[0]
zip_pattern = "{}.*$".format(just_fn)

In [58]:
search_all_output = []
# Gather all files for the zip
for root, dirs, files in os.walk(RESULTS_DIR):
    ## this comes up blank, don't know why
    # search_all_output = [fn for fn in files if re.match(zip_pattern, fn)]
    
    for fn in files:
        if re.match(zip_pattern, fn):
            search_all_output.append(fn)

In [59]:
# Remove CSV and other ZIP placeholder
zip_shapefile_list = []
for fn in search_all_output:
    if not fn.endswith('.zip') and not fn.endswith('.csv'):
        zip_shapefile_list.append(fn)

<b> ZIP files </b>

In [60]:
for fn in zip_shapefile_list:
    my_zipped_shape.write(os.path.join(RESULTS_DIR, fn))

<b> Publish to ESRI API </b>

In [61]:
try:
    my_output_shape = gis.content.add({'type': 'Shapefile'}, my_zipped_shape.filename)
    my_output_shape_BLC = my_output_shape.publish()

except RuntimeError:
    # choose query on Username
    query_user = "owner:{}".format(gis.users.me.username)
    
    # choose query on filename <title>
    my_published_zip = os.path.basename(output_zip).split('.')[0]
    query_title = "title:{}".format(my_published_zip)
    
    # search for published file
    my_files = gis.content.search(query=query_title, item_type='Feature Service')
    my_output_shape_BLC = my_files[0]
    
except Exception as e:
    print(e)

Item 'employee_trips_blc72.zip' already exists.


In [62]:
my_output_shape_BLC

<Item title:"employee_trips_blc72" type:Feature Layer Collection owner:blc72_pennstate>

In [66]:
myMap = gis.map('Pennsylvania')
myMap.add_layer(my_output_shape_BLC, {})

# Extra Effort:

<b> Find the longest and shortest distance trip from results
NOT WORKING</b>

In [128]:
#my_trips_df['Distance'] = my_trips_df['geometry'].apply(lambda x: osr.getLength(x))

# from shapely.geometry import LineString
# from math import radians, cos, sin, asin, sqrt

# # Calculates distance between 2 GPS coordinates
# def haversine(lat1, lon1, lat2, lon2):
#     """
#     Calculate the great circle distance between two points 
#     on the earth (specified in decimal degrees)
#     """
#     # convert decimal degrees to radians 
#     lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

#     # haversine formula 
#     dlon = lon2 - lon1 
#     dlat = lat2 - lat1 
#     a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
#     c = 2 * asin(sqrt(a)) 
#     r = 3956 # Radius of earth in kilometers. Use 3956 for miles
#     return c * r


# for row in my_trips_df.index:
#     line = my_trips_df.loc[row, 'geometry']
#     route = my_trips_df.loc[row, 'Route']
    
#     print(type(line))
#     route_stops = iter(line)
#     print(route_stops)
#     print(route_stops[0])

#     route_stops = len(route.split(';')) - 1
#     distance = 0
#     for i in range(0, route_stops):
#         pointN = line.coords[i]
#         pointM = line.coords[i + 1]
#         distance += haversine(pointN[0], pointN[1], pointM[0], pointM[1])

#         print (distance)



    


If file has already been Published, pull from my shared repository

In [64]:
neCities_zip = IN_filepath + my_neCities_file + '.zip'
try:
    neCities = gis.content.add({'type': 'Shapefile'}, neCities_zip)
    my_neCities = neCities.publish()
except RuntimeError:
    query_user = "owner:{}".format(gis.users.me.username)
    query_title = "title:{}".format(my_neCities_file)
    my_files = gis.content.search(query=query_title, item_type='Feature Service')
    my_neCities = my_files[0]
    
except Exception as e:
    print(e)


In [67]:
myMap.add_layer(my_neCities, {})
myMap

MapView(layout=Layout(height='400px', width='100%'))

In [211]:
from math import radians, cos, sin, asin, sqrt


top_stats = ['Name', 'total_trips', 'total_travel_days',\
             'total_cities', 'cities_visited', 'total_time_span',\
             'longest_trip_days','shortest_trip_days',\
             'most_stops_in_single_trip',\
             'total_distance_mi']

top_df = pd.DataFrame(columns=top_stats)
ndx = 0
for name in employeeList:
    my_emp_stats = {'Name':name}
    analyze_df = my_trips_df[my_trips_df.Name == name]
    my_emp_stats['total_trips'] = len(analyze_df)
    total_travel_days = analyze_df.Duration.sum()
    my_emp_stats['total_travel_days'] = total_travel_days
    
    all_cities = []
    for row in analyze_df.index:
        this_route = analyze_df.loc[row, 'Route'].split(';')
        for stop in this_route:
            if stop not in all_cities:
                all_cities.append(stop)
    my_emp_stats['total_cities'] = len(all_cities)
    my_emp_stats['cities_visited'] =";".join(all_cities)
    
    my_emp_stats['total_time_span'] = (max(analyze_df['End']) - min(analyze_df['Start'])).days
    
    my_emp_stats['longest_trip_days'] = max(analyze_df.Duration)
    my_emp_stats['shortest_trip_days'] = min(analyze_df.Duration)
    
    my_emp_stats['most_stops_in_single_trip'] = len(max(analyze_df.Route).split(';'))
    
    
    # Calculating the distance will require adding the start city to
    # the end of the list to return to home: geom_wrh (with return home)
    distance = 0
    for row in analyze_df.index:
        route = analyze_df.loc[row, 'Route'].split(';')
        route.append(route[0])
        route_wrh = ";".join(route)
        wkt_wrh = get_coordinates(route_wrh, ne_Cities_df)
        geom_wrh = ogr.CreateGeometryFromWkt(wkt_wrh)
        
        a = math.radians(geom_wrh.Length())
        r = 3956 # Radius of earth. Use 3956 for miles
        distance += (a * r)
        
    
    my_emp_stats['total_distance_mi'] = round(distance,1)
    new_line_df = pd.DataFrame(my_emp_stats, columns=top_stats, index=[ndx])
    ndx += 1
    top_df = pd.concat([top_df, new_line_df], axis=0)



In [212]:
top_df['avg_distance_per_travel_day (total_distance/total_travel_days)'] = top_df[['total_travel_days','total_distance_mi']].apply(lambda x: round(x[1]/x[0], 1), axis = 1)
top_df['avg_days_per_trip (total_travel_days/total_trips)'] =  top_df[['total_travel_days','total_trips']].apply(lambda x: round(x[0]/x[1], 1), axis = 1)
top_df['travel_time_PCT (total_travel_days/total_time_span)'] = top_df[['total_travel_days', 'total_time_span']].apply(lambda x: round(100*x[0]/x[1], 1), axis = 1)
top_df['trip count ratio (one trip):(total_time_span/total_trips)'] = top_df[['total_trips', 'total_time_span']].apply(lambda x: round(x[1]/x[0], 1), axis=1)
top_df

,Name,total_trips,total_travel_days,total_cities,cities_visited,total_time_span,longest_trip_days,shortest_trip_days,most_stops_in_single_trip,total_distance_mi,avg_distance_per_travel_day (total_distance/total_travel_days),avg_days_per_trip (total_travel_days/total_trips),travel_time_PCT (total_travel_days/total_time_span),trip count ratio (one trip):(total_time_span/total_trips)
0,"Jones, Frank",3,32,10,Harrisburg;Augusta;Philadelphia;Boston;Altoona...,266,17,5,5,4526.2,141.4,10.7,12.0,88.7
1,"Brown, Justine",6,74,13,Pittsburgh;Syracuse;Columbus;Boston;Albany;Scr...,254,19,5,6,9215.3,124.5,12.3,29.1,42.3
2,"Samulson, Roger",5,56,15,Boston;Syracuse;Portland;Altoona;Augusta;Burli...,230,18,7,4,9405.2,168.0,11.2,24.3,46.0


In [214]:
analysis_fn = os.path.join(RESULTS_DIR, 'analysis.csv')
top_df.to_csv(analysis_fn)